This Jupyter Notebook is a basic guide to exploratory data analysis using Python Language, SQL Language and SQLite Database, provided by the "Python Fundamentals of Data Analysis 3.0" course offered by the Data Science Academy. Actual publicly available data, movie data on IMDB will be used.
By applying Exploratory Data Analysis it will be possible to answer these 10 questions:

1- What are the Most Common Movie Categories on IMDB?
2- What is the number of titles by genre?
3- What is the Median Rating of Films by Genre?
4- What is the Median Evaluation of Films Regarding the Year of Premiere?
5- What is the number of films evaluated by genre in relation to the year of debut?
6- What is the longest running movie? Calculate the percentiles.
7- What is the relationship between duration and genre?
8- What is the number of films produced per country?
9- What are the Top 10 Best Movies?
10- What are the Top 10 Worst Movies?

In [8]:
from platform import python_version
print('Versão da linguagem python usada neste jupyter notebook', python_version()) 

Versão da linguagem python usada neste jupyter notebook 3.9.7


In [9]:
#instalando o pacote destinado a trazer de forma organizada os dados contidos na plataforma IMDB
!pip install imdb-sqlite

In [10]:
#instalando o pacote que decodifica o padrão ISO as siglas nos nomes dos países
!pip install pycountry 

In [11]:
#Importando pacotes necessários para as análises, visualizaçoes e outros.

#expressão regulares
import re 
#medir tempo de execução ou manipular datas
import time 
#trbalhar com o banco de dados SQLite
import sqlite3 
#decodificar nom de países
import pycountry 
#manipulação de dados
import numpy as np 
#manipulação de dados
import pandas as pd 
#vizualização de dados
import matplotlib.pyplot as plt
#vizualização de dados
import seaborn as sns 
#vizualização de dados
from matplotlib import cm 
#machine learning
from sklearn.feature_extraction.text import CountVectorizer 
#filtrar warnings
import warnings 
warnings.filterwarnings("ignore") 
#define um padrão de fundo branco para os gráficos
sns.set_theme(style = "whitegrid") 

Carregando os dados

In [12]:
%%time
#download dos arquivos com os dados e criar um banco de dados
!imdb-sqlite 

Wall time: 464 ms


2022-03-02 13:58:51,868 DB already exists: (imdb.db). Refusing to modify. Exiting


In [13]:
#conectar ao banco de dados criado através do pacote imdb 
conn = sqlite3.connect("imdb.db")

In [14]:
#extrair a lista de tabelas que estão no banco
tabelas = pd.read_sql_query("SELECT name AS 'Table_Name' FROM sqlite_master WHERE type = 'table'",conn)

In [15]:
#verificar o tipo do objeto
type(tabelas)

pandas.core.frame.DataFrame

In [16]:
#vizualizar os resultados
tabelas.head()

,Table_Name
0,people
1,titles
2,akas
3,crew
4,episodes


In [17]:
#convertendo a tabela de nomes e uma lista para facilitar a vizualição
tabelas = tabelas["Table_Name"].values.tolist()

tabelas

['people', 'titles', 'akas', 'crew', 'episodes', 'ratings']

In [18]:
#extrair o schema de cada tabela para conhecer os tipos de dados  e como eles estão organizados

for tabela in tabelas:
    #"PRAGMA TABLE_INFO({})" usado no SQLite extrai as informações da tabela 
    consulta= "PRAGMA TABLE_INFO({})".format(tabela)
    resultado = pd.read_sql_query(consulta, conn)
    print("Esquema da tabela: ",tabela)
    #display vizualiza o resultado
    display(resultado)
    print("-"*100)
    print("\n")
    

Esquema da tabela:  people


,cid,name,type,notnull,dflt_value,pk
0,0,person_id,VARCHAR,0,None,1
1,1,name,VARCHAR,0,None,0
2,2,born,INTEGER,0,None,0
3,3,died,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela:  titles


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,type,VARCHAR,0,None,0
2,2,primary_title,VARCHAR,0,None,0
3,3,original_title,VARCHAR,0,None,0
4,4,is_adult,INTEGER,0,None,0
5,5,premiered,INTEGER,0,None,0
6,6,ended,INTEGER,0,None,0
7,7,runtime_minutes,INTEGER,0,None,0
8,8,genres,VARCHAR,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela:  akas


...

----------------------------------------------------------------------------------------------------


Esquema da tabela:  episodes


,cid,name,type,notnull,dflt_value,pk
0,0,episode_title_id,INTEGER,0,None,0
1,1,show_title_id,INTEGER,0,None,0
2,2,season_number,INTEGER,0,None,0
3,3,eposide_number,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


Esquema da tabela:  ratings


,cid,name,type,notnull,dflt_value,pk
0,0,title_id,VARCHAR,0,None,1
1,1,rating,INTEGER,0,None,0
2,2,votes,INTEGER,0,None,0


----------------------------------------------------------------------------------------------------


